In [ ]:
from datetime import datetime
import os
import sys

In [ ]:
# script_dir = os.path.dirname(__file__)
script_dir = os.path.abspath('')
util_dir = os.path.join(script_dir, '..', 'Common_Utils')
sys.path.append(util_dir)
import file_utilities as futil

In [ ]:
input_path = r"..\Files\Input_Files\Table_Column_Info.csv"
output_path = "..\Files\Output_Files\Table_DDL"
contents = futil.read_file(input_path)
futil.make_directory_if_not_exists(output_path)

output_content_list = []
table_name = ''
column_name = ''
data_type = ''
header_rec = True

In [ ]:
def write_sql_file(table_name, output_content_list):
        output_full_path = f"{output_path}\{table_name}.sql"
        sqlfile_contents = '\n'.join(output_content_list)
        futil.write_file(output_full_path, sqlfile_contents)

In [ ]:
# Generate individual ddl files with parameterization
dt1 = datetime.now()
lines = contents.split("\n")
for index, line in enumerate(lines):
    if line.strip() == '' or (line.strip() != '' and index == 0 and header_rec == True) :
        continue
    else:
        fields = line.split(",")
        if (table_name != fields[0].strip().lower()):
            if table_name != '':
                output_content_list.append(');')
                write_sql_file(table_name, output_content_list)

            output_content_list = []
            table_name = fields[0].strip().lower()
            output_content_list.append('CREATE TABLE IF NOT EXISTS {{ params.param_stage_dataset_name }}.' + table_name)
            output_content_list.append('(')
        
        column_name = fields[1].strip().lower()
        data_type = fields[2].strip().lower()
        if column_name == 'group': # Keywords needs to be in back ticks
            column_name = f'`{column_name}`'
        output_content_list.append('\t' + column_name + ' ' + data_type + ',')

if table_name != '':
    output_content_list.append(');')
    write_sql_file(table_name, output_content_list)

dt2 = datetime.now()
print(dt2-dt1)